In [1]:
%load_ext jupyter_black

In [2]:
import json
from glob import glob
from pathlib import Path
from typing import NewType, Iterable, Callable

import nvector
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import nvector as nv

# ml
import tensorflow
from sklearn.manifold import LocallyLinearEmbedding

# plotting
import matplotlib.pyplot as plt

# number of seconds in 2 mins
TWO_MINS = 120.0
idx: slice = pd.IndexSlice

FeatureCollection = NewType("FeatureCollection", dict)

wgs84 = nv.FrameE(name="WGS84")

all_files = sorted(glob("/workspaces/sppp/data/probsevere/*.json"))

In [3]:
# data transformation and initial loading
def open_file(filepath: Path) -> FeatureCollection:
    with filepath.open("rb") as fin:
        return json.load(fin)


def to_dataframe(fc: FeatureCollection) -> pd.DataFrame:
    df = gpd.GeoDataFrame.from_features(fc["features"])
    df["validTime"] = pd.to_datetime(fc["validTime"], format="%Y%m%d_%H%M%S %Z")
    df["CENTROID"] = df["geometry"].centroid

    def ecef_vector():
        for point in df["geometry"].centroid:
            geo_point = wgs84.GeoPoint(
                longitude=point.x, latitude=point.y, degrees=True
            )
            yield geo_point.to_ecef_vector()

    df["ECEF_VECTOR"] = tuple(ecef_vector())

    df = df.set_index(["validTime", "ID"])
    return df
    # df =


def to_midf() -> pd.DataFrame:
    def generate():
        for file in all_files:
            fc = open_file(Path(file))
            yield to_dataframe(fc)

    return pd.concat(generate())


midf = to_midf()
midf

geometry  \
validTime                 ID                                                         
2021-10-11 00:00:53+00:00 89234  POLYGON ((-96.06000 38.40000, -95.99000 38.400...   
                          89321  POLYGON ((-80.66000 24.76000, -80.62000 24.760...   
                          89467  POLYGON ((-96.98000 36.80000, -96.87000 36.800...   
                          89470  POLYGON ((-75.15000 35.44000, -75.11000 35.440...   
                          89519  POLYGON ((-98.29000 35.59000, -98.27000 35.590...   
...                                                                            ...   
2021-10-11 01:00:58+00:00 90389  POLYGON ((-97.10000 37.72000, -97.05000 37.720...   
                          90390  POLYGON ((-98.03000 34.15000, -97.99000 34.100...   
                          90391  POLYGON ((-98.87000 31.84000, -98.85000 31.830...   
                          90392  POLYGON ((-99.03000 31.68000, -99.01000 31.680...   
                          90393  POLYGON ((-99.35000 31.38000, -99.31000 31.380...   

                                MUCAPE MLCAPE MLCIN EBSHEAR SRH01KM  \
validTime                 ID                                          
2021-10-11 00:00:53+00:00 89234    485      0     0    48.6      63   
                          89321   2691   2179    -2    22.9      14   
                          89467   1367    937   -19    61.9     148   
                          89470   2493   2076     0    24.0      19   
                          89519   2035   1656   -17    74.6     197   
...                                ...    ...   ...     ...     ...   
2021-10-11 01:00:58+00:00 90389    576      0     0    52.1      74   
                          90390   2113   2067   -31    59.3     192   
                          90391   2362   1910   -22    67.5     197   
                          90392   2208   1744   -15    66.8     170   
                          90393   1812   1428   -32    58.1      88   

                                MEANWIND_1-3kmAGL  MESH VIL_DENSITY  \
validTime                 ID                                          
2021-10-11 00:00:53+00:00 89234              12.9  0.25        1.67   
                          89321               4.0  0.43        1.60   
                          89467              17.6  1.43        2.85   
                          89470              16.3  0.05        0.62   
                          89519              26.4  2.19        3.29   
...                                           ...   ...         ...   
2021-10-11 01:00:58+00:00 90389               8.4  0.00        0.52   
                          90390              36.4  0.25        0.88   
                          90391              32.8  0.04        0.77   
                          90392              32.2  0.59        1.75   
                          90393              31.5  0.48        1.68   

                                FLASH_RATE  ... PWAT CAPE_M10M30  LJA  SIZE  \
validTime                 ID                ...                               
2021-10-11 00:00:53+00:00 89234         10  ...  1.5         129  0.0   515   
                          89321          7  ...  1.4         668  0.0    45   
                          89467         60  ...  1.7         319  1.2   355   
                          89470          0  ...  2.3         420  0.0    50   
                          89519        181  ...  1.9         603  1.2  1685   
...                                    ...  ...  ...         ...  ...   ...   
2021-10-11 01:00:58+00:00 90389          0  ...  1.5         160  0.0   124   
                          90390          4  ...  1.9         704  0.0    48   
                          90391          0  ...  1.6         635  0.0    91   
                          90392         12  ...  1.5         594  0.0   210   
                          90393         15  ...  1.2         484  0.0   415   

                                        AVG_BEAM_HGT MOTION_EAST MOTION_SOUTH  

In [4]:
# helper function
from sklearn.manifold import LocallyLinearEmbedding
from sppp.transfer.const import GridSpace


grid = GridSpace()


def embed(df: pd.DataFrame, grid: GridSpace) -> pd.DataFrame:

    lle = LocallyLinearEmbedding(n_components=1, n_neighbors=10)
    fresh = df[["PS", "MOTION_EAST", "MOTION_SOUTH"]].copy()
    cent = df["geometry"].centroid
    fresh["STAB"] = lle.fit_transform(midf[["MUCAPE", "MLCAPE", "MLCIN"]])
    fresh = fresh.astype(np.float32)

    fresh["X"] = np.argmin(abs(cent.x.values[:, np.newaxis] - grid.x), axis=1)
    fresh["Y"] = np.argmin(abs(cent.y.values[:, np.newaxis] - grid.y), axis=1)

    return fresh


fresh = midf.pipe(embed, grid).loc[idx[:, "89519"], :]
fresh

,,PS,MOTION_EAST,MOTION_SOUTH,STAB,X,Y
validTime,ID,,,,,,
2021-10-11 00:00:53+00:00,89519,100.0,12.789000,-8.273,0.000160,3176,1987
2021-10-11 00:02:40+00:00,89519,100.0,12.779000,-8.910,0.000105,3179,1984
2021-10-11 00:04:47+00:00,89519,100.0,14.756000,-7.504,-0.000145,3186,1979
2021-10-11 00:06:55+00:00,89519,100.0,13.202000,-6.609,-0.000257,3187,1978
2021-10-11 00:08:38+00:00,89519,100.0,15.162000,-7.446,-0.000284,3189,1976
2021-10-11 00:10:59+00:00,89519,100.0,12.151000,-6.134,-0.000311,3192,1978
2021-10-11 00:12:40+00:00,89519,100.0,16.190001,-6.856,-0.000284,3195,1977
2021-10-11 00:14:44+00:00,89519,100.0,16.011000,-5.761,-0.000030,3198,1977
2021-10-11 00:16:50+00:00,89519,100.0,16.136999,-5.559,-0.000072,3201,1975


In [5]:
# machine learning support classes
from tensorflow import keras
from keras.engine.sequential import Sequential


class State:
    __has_state: bool = False

    def __init__(self):
        self.__latests: pd.DataFrame = None
        self.__state: pd.DataFrame = None

    def __repr__(self) -> str:
        return self.__state.__repr__()

    @property
    def latests(self) -> pd.DataFrame:
        return self.__latests

    def set_storm(self, df: pd.DataFrame) -> None:

        self.__latests = df

        if not self.__has_state:
            self.__has_state = True
            self.__state = df
        else:
            old = self.__state
            self.__state = pd.concat([old, df])

    def iterstorms(self):
        df = self.__state.iloc[-2:].groupby("ID")
        for id, x in self.__state.iloc[-2:].groupby("ID"):
            yield x

    def has_hist(self) -> bool:
        return isinstance(self.__state, pd.DataFrame)

    @property
    def frame(self) -> pd.DataFrame:
        return self.__state


def build_model(frame_b: pd.DataFrame) -> Sequential:
    model = keras.models.Sequential(
        [
            keras.layers.Dense(
                22.5,
                activation="elu",
                input_shape=frame_b.shape,
            ),
            keras.layers.Dense(
                22.5,
                activation="sigmoid",
            ),
            keras.layers.Dense(3),
        ]
    )
    return model


# grid.x
zeros = pd.DataFrame(
    columns=np.arange(len(grid.x)), index=np.arange(len(grid.y))
).fillna(0)


def make_reward_map():

    a = list(np.linspace(0, 0.25, 5))
    a2 = np.array(a + [0.5] + list(reversed(a)))
    return a2 + a2[:, np.newaxis]

In [65]:
import tensorflow as tf
import gym

from sppp.transfer.funcs import mask_frames_by_id
from sppp.transfer.const import GridSpace

grid = GridSpace()


class SPPPEnv(gym.Env):
    def __init__(self, state: "State") -> None:
        self.state = state

    def _compute_reward(self) -> int:
        return 1

    def step(self, action: np.ndarray):
        # action is produced by DQN, action is discrete
        # self.cache.move(action)
        # compute reward based on state(position) of the car
        # storm_state = self.car_agent.getCarState()
        # reward = self._compute_reward(storm_state)
        # # check if the episode is done
        # car_controls = self.car_agent.getCarControls()
        # done = self._isDone(storm_state, car_controls, reward)
        # # log info
        # info = {}
        # # observation is RGB image from car's camera
        # observation = self.car_agent.observe()
        observation = self.state.latests
        reward = self._compute_reward()
        done = False
        info = {}
        return observation, reward, done, info

    @property
    def observation_space(self) -> pd.DataFrame:
        return self.state.latests


def iterframe(df: pd.DataFrame) -> Iterable[tuple[pd.Timestamp, pd.DataFrame]]:
    yield from df.groupby("validTime")


n_inputs = 4
state = State()
env = SPPPEnv(state)
loss = keras.losses.binary_crossentropy
reward_map = make_reward_map()


n_outputs = 5
box_shape = 5


def reward_matrix(x: int, y: int, box_size: int = 5):
    gamefield = (
        zeros.loc[
            y - box_size : y + box_size,
            x - box_size : x + box_size,
        ].copy()
        + reward_map
    )
    return gamefield.values


transition_probs = [[0, 0, 1] * 20] * 101
possible_actions = [[0, 1, 2], [0, 2], [1]] * 101  # [list(range(-5, 5))] * 101


def swp_class_ploicy(
    state: int,  # probiblity of severe weather
    epsilon=0,
):
    if np.random.rand() < epsilon:
        return np.random.randint()
    # v = model.predict(state[np.newaxis])
    return np.random.choice(possible_actions[state])


def training_step(
    env: SPPPEnv,
    obs: np.ndarray,
    model: Sequential,
    loss_fn: Callable[[any], any],
):
    """policy gradient"""
    # with tf.GradientTape() as tape:
    #     left_prob = model(obs[np.newaxis, :])
    #     action = tf.random.uniform([1, 1]) > left_prob

    #     y_target = tf.constant([[1.0]]) - tf.cast(action, tf.float32)

    #     loss = tf.reduce_mean(loss_fn(y_target, left_prob))

    # grads = tape.gradient(loss, model.trainable_variables)
    # return action.numpy()


if __name__ == "__main__":
    for vt, df in iterframe(fresh):
        track_rewards = 0
        obs = env.reset()
        # evaluate prediction
        if state.has_hist():
            # there is existing storm information normalize frame_a and frame_b by the the in's in the index
            frame_a, frame_b = mask_frames_by_id(state.latests, df)
            # assert that the frames are of an equal shape
            assert frame_a.shape == frame_b.shape
            bg = frame_b[["MOTION_EAST", "MOTION_SOUTH", "STAB"]].values

            # x = play_one_step(env, bg, model, loss)
            # obs, reward, done, info = env.step(frame_b[["MOTION_EAST", "MOTION_SOUTH"]])
            with tf.GradientTape() as tape:
                for x, s in frame_b.iterrows():
                    model = build_model(s)
                    action = swp_class_ploicy(int(tup.PS))
                # ...
            # for tup in frame_b.itertuples():
            #         reward = reward_matrix(tup.X, tup.Y)
            #         model.predict()
            # training_step(env, frame_b[["X", "Y"]].values, model, loss)
            # prob = step(0, action)

        state.set_storm(df)
# prob
# frame_b
s
# np.random.choice(a=[1], p=prob)
# gamefield

PS               100.000000
MOTION_EAST       12.542000
MOTION_SOUTH      -3.614000
STAB               0.001055
X               3254.000000
Y               1968.000000
Name: (2021-10-11 01:00:58+00:00, 89519), dtype: float64

In [25]:
np.random.choice(a=[1], p=[1])
# gamefield
# frame_b

1

In [7]:
df = state.frame.copy()
df.iloc[0]
# df["ECEF_VECTOR"] + (df[["X", "Y"]] * TWO_MINS).stack().values[:, np.newaxis]

PS               100.00000
MOTION_EAST       12.78900
MOTION_SOUTH      -8.27300
STAB               0.00016
X               3176.00000
Y               1987.00000
Name: (2021-10-11 00:00:53+00:00, 89519), dtype: float64